<a href="https://colab.research.google.com/github/kavnthir/ECE4994/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet norse

     |████████████████████████████████| 109 kB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [2]:
import torch
import norse
import numpy as np
import matplotlib.pyplot as pl